# Linear Modeling

The goal of this lab is to design a class for linear regression, using no classes or functions from Scikit-Learn.

Ensure your class satisfies:
- Includes a class method to fit the model to a Pandas dataframe $X$ and a Pandas series $y$
- Includes a class method to solve for the optimal coefficients
- Includes a class method to make predictions, given a new matrix $\hat{X}$
- Does not invert any matrices explicitly. Instead, solve the normal equations using `np.lingalg.solve`.
- It can be instructed to automatically perform a train-test split and return performance metrics on the test set.
- It can provide metrics including SSE, MSE, RMSE, and $R^2$.

Before programming your class, consider the following questions and record the answers:
- How does your class handle categorical data? How does Sci-kit do it?
- **My Class**:
    
    - Uses `pd.get_dummies(X, drop_first=True)` to manually one-hot encode categorical columns.
        
    - `drop_first=True` avoids multicollinearity by removing one dummy variable per feature.
        
- **Scikit-Learn**:
    
    - Uses `OneHotEncoder` (typically inside a `ColumnTransformer`) for flexible handling.
        
    - More powerful: can handle unseen categories, return sparse matrices, etc.
- How does your class handle missing data? How does Sci-kit do it?
- **My Class**:
    
    - Drops all rows with missing values (`.dropna()`).
        
    - Simple but could lead to significant data loss.
        
- **Scikit-Learn**:
    
    - Provides `SimpleImputer` and `IterativeImputer` for flexible imputation (mean, median, constant, MICE).
        
    - Can impute missing data without dropping rows.
        
- Does your class have any methods for creating polynomial expansions or otherwise transforming data? How does Sci-kit do it?
- **My Class**:
    
    - Does **not** support polynomial features or nonlinear transformations out of the box.
        
    - Could be extended to add polynomial features manually or via a helper.
        
- **Scikit-Learn**:
    
    - Supports polynomial expansion using `PolynomialFeatures` from `sklearn.preprocessing`.
        
    - Integrates smoothly into pipelines for feature engineering.
- How does your class handle the bias/intercept/constant? How does Sci-kit do it?
- **My Class**:
    
    - Manually inserts an `Intercept` column with value 1.0 at index 0 of `X`.
        
    - The normal equation then computes the bias term correctly.
        
- **Scikit-Learn**:
    
    - Uses `fit_intercept=True` by default, which automatically adds the intercept during fitting.
- What output do you automatically provide to the user? Why? How does Sci-kit do it?
- **My Class**:
    
    - Automatically provides:
        
        - `SSE`: Sum of Squared Errors
            
        - `MSE`: Mean Squared Error
            
        - `RMSE`: Root Mean Squared Error
            
        - `R²`: Coefficient of Determination
            
    - **Why**: These are standard evaluation metrics for regression and offer intuitive diagnostics for model performance.
        
- **Scikit-Learn**:
    
    - Provides:
        
        - `.score()` returns R²
            
        - Additional metrics like `mean_squared_error`, `mean_absolute_error`, etc., are in `sklearn.metrics`.
            
- Are you including any tools for statistical inference? How does Sci-kit do it?

- **My Class**:
    
    - Does **not** include p-values, confidence intervals, standard errors, or hypothesis tests.
        
    - These would require further statistical modeling or integration with `statsmodels`.
        
- **Scikit-Learn**:
    
    - Also **does not** provide statistical inference out of the box.
        
    - For statistical inference, the go-to library is typically `statsmodels`, not `scikit-learn`.

In order to measure how long it takes to run code, you can `import time`, and 

```
start = time.time()
<expressions and code>
finish = time.time()
runtime = start-finish
```

For the `heart_hw.csv` and `cars_hw.csv` data in the assignment folder, run some regressions and compare the performance of your class with Sci-Kit's linear regression model. Do you get the same answers for the optimal coefficients, SSE, and $R^2$? Which one runs faster?



In [18]:
# Linear Regression from Scratch - No Scikit-Learn
import pandas as pd
import numpy as np
import time
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error

# --------------------------
# Custom Linear Regression Class
# --------------------------
class MyLinearRegression:
    def __init__(self):
        self.coef_ = None
        self.fitted = False

    def _prepare_X(self, X):
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X)
        X = pd.get_dummies(X, drop_first=True)
        X = X.dropna()
        X.insert(0, 'Intercept', 1.0)
        return X.astype(float)


    def fit(self, X, y):
        X = self._prepare_X(X)
        y = y.loc[X.index].astype(float)
        self.feature_names = X.columns
        XtX = X.T @ X
        Xty = X.T @ y
        self.coef_ = np.linalg.solve(XtX, Xty)
        self.fitted = True

    def predict(self, X):
        if not self.fitted:
            raise ValueError("Model not fitted.")
        X = pd.get_dummies(X, drop_first=True)
        X = X.dropna()
        X.insert(0, 'Intercept', 1.0)
        for col in self.feature_names:
            if col not in X.columns:
                X[col] = 0
        X = X[self.feature_names]
        return X @ self.coef_

    def score_metrics(self, y_true, y_pred):
        residuals = y_true - y_pred
        sse = np.sum(residuals ** 2)
        mse = np.mean(residuals ** 2)
        rmse = np.sqrt(mse)
        r2 = 1 - sse / np.sum((y_true - np.mean(y_true)) ** 2)
        return {"SSE": sse, "MSE": mse, "RMSE": rmse, "R^2": r2}

    def train_test_split_fit_score(self, X, y, test_size=0.2, random_state=42):
        X = pd.get_dummies(X, drop_first=True).dropna()
        y = y.loc[X.index]
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )
        self.fit(X_train, y_train)
        y_pred = self.predict(X_test)
        return self.score_metrics(y_test, y_pred)

# --------------------------
# Load Data
# --------------------------
heart_df = pd.read_csv("/home/hanyan/dev/linearModels/assignment/data/heart_hw.csv")
cars_df = pd.read_csv("/home/hanyan/dev/linearModels/assignment/data/cars_hw.csv")

# --------------------------
# Run on heart_hw.csv
# --------------------------
start = time.time()
my_model_heart = MyLinearRegression()
heart_X = heart_df[['age', 'transplant']]
heart_y = heart_df['y']
heart_metrics = my_model_heart.train_test_split_fit_score(heart_X, heart_y)
my_time_heart = time.time() - start

start = time.time()
X = pd.get_dummies(heart_X, drop_first=True).dropna()
y = heart_y.loc[X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sk_model = LinearRegression().fit(X_train, y_train)
y_pred = sk_model.predict(X_test)
sk_metrics_heart = {
    "SSE": np.sum((y_test - y_pred) ** 2),
    "MSE": mean_squared_error(y_test, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "R^2": r2_score(y_test, y_pred),
}
sk_time_heart = time.time() - start

print("Heart Dataset")
print("MyLinearRegression:", heart_metrics, f"Time: {my_time_heart:.4f}s")
print("Scikit-Learn:", sk_metrics_heart, f"Time: {sk_time_heart:.4f}s\n")

# --------------------------
# Run on cars_hw.csv
# --------------------------
cars_X = cars_df.drop(columns=['Unnamed: 0', 'Price'])
cars_y = cars_df['Price']

start = time.time()
my_model_cars = MyLinearRegression()
cars_metrics = my_model_cars.train_test_split_fit_score(cars_X, cars_y)
my_time_cars = time.time() - start

start = time.time()
X = pd.get_dummies(cars_X, drop_first=True).dropna()
y = cars_y.loc[X.index]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
sk_model_cars = LinearRegression().fit(X_train, y_train)
y_pred = sk_model_cars.predict(X_test)
sk_metrics_cars = {
    "SSE": np.sum((y_test - y_pred) ** 2),
    "MSE": mean_squared_error(y_test, y_pred),
    "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
    "R^2": r2_score(y_test, y_pred),
}
sk_time_cars = time.time() - start

print("Cars Dataset")
print("MyLinearRegression:", cars_metrics, f"Time: {my_time_cars:.4f}s")
print("Scikit-Learn:", sk_metrics_cars, f"Time: {sk_time_cars:.4f}s")


Heart Dataset
MyLinearRegression: {'SSE': 3.77320941682889, 'MSE': np.float64(0.1796766388966138), 'RMSE': np.float64(0.42388281269310013), 'R^2': np.float64(0.19145512496523776)} Time: 0.0108s
Scikit-Learn: {'SSE': np.float64(3.77320941682889), 'MSE': 0.1796766388966138, 'RMSE': np.float64(0.42388281269310013), 'R^2': 0.19145512496523776} Time: 0.0084s

Cars Dataset
MyLinearRegression: {'SSE': 4040312067805.849, 'MSE': np.float64(20613837080.642086), 'RMSE': np.float64(143575.19660666352), 'R^2': np.float64(0.8202733305969847)} Time: 0.0068s
Scikit-Learn: {'SSE': np.float64(4040312067762.737), 'MSE': 20613837080.422127, 'RMSE': np.float64(143575.1966058975), 'R^2': 0.8202733305989025} Time: 0.0060s


## Timeing with heart

In [ ]:
import pandas as pd

heart_df = pd.read_csv("heart_hw.csv")

X_heart = heart_df[['age', 'transplant']].copy()
y_heart = heart_df['y'].copy()

X_heart = X_heart.dropna()
y_heart = y_heart[X_heart.index]

# One-hot encode 'transplant' (creates new columns 'transplant_control','transplant_treatment', etc.)
X_heart = pd.get_dummies(X_heart, columns=['transplant'], drop_first=True)

# Now X_heart is numeric (likely float64).
print(X_heart.dtypes)

model_heart_custom = MyLinearRegression(fit_intercept=True, test_size=0.0)
model_heart_custom.fit(X_heart, y_heart)


ValueError: invalid literal for int() with base 10: 'control'